"[Список стран по ожидаемой продолжительности жизни](https://ru.wikipedia.org/wiki/Список_стран_по_ожидаемой_продолжительности_жизни)"<br>
"[Продолжительность жизни в субъектах Российской Федерации](https://ru.wikipedia.org/wiki/Продолжительность_жизни_в_субъектах_Российской_Федерации)"<br>
[fedstat.ru](https://www.fedstat.ru/indicator/31293) (ЕМИСС - Единая межведомственная информационно-статистическая система)<br>
[Население субъектов РФ](https://ru.wikipedia.org/wiki/Население_субъектов_Российской_Федерации)

In [1]:
import pandas as pd
import numpy as np
import math
import re

In [2]:
pd.options.display.max_rows = 200
pd.options.display.max_columns = 50

In [3]:
# load stats about longevity per year
df = pd.read_excel('data/Rosstat_by_year_and_region.xls', skiprows=2)

print(df.shape)
df.head(2)

(108, 35)


,Unnamed: 0,Unnamed: 1,Unnamed: 2,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,Российская Федерация,Оба пола,все население,69.2,68.9,67.8,65.0,63.9,64.5,65.8,66.7,67.1,65.9,65.3,65.2,65.0,64.9,65.3,65.4,66.7,67.6,68.0,68.8,68.9,69.83,70.24,70.76,70.93,71.39,71.87,72.70,72.91,73.34,71.54,70.06
1,Центральный федеральный округ,Оба пола,все население,69.5,69.2,68.3,65.6,64.2,64.9,66.5,67.4,67.6,66.4,66.1,65.8,65.6,65.7,66.3,66.5,67.6,68.4,68.8,69.7,69.9,71.19,71.43,71.93,72.10,72.72,73.07,73.89,74.01,74.54,72.57,70.85


In [4]:
# Clean names of regions from surrounding spaces
df['Unnamed: 0'] = df['Unnamed: 0'].map(lambda st: st.strip())

# convert region column to index
df.set_index('Unnamed: 0', inplace=True)

# for brevityh of display remove name of index column.
df.index.name = ''

# sort index to speed up further transformations
df.sort_index(inplace=True)

# patch for Ingushetia for 1990-1992
# df.loc['Республика Ингушетия', '1990':'1992'] = df.loc['Чеченская и Ингушская Республики', '1990':'1992']

print(df.shape)
df.head(2).fillna(' ')

(108, 34)


,Unnamed: 1,Unnamed: 2,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Агинский Бурятский округ (Забайкальский край),Оба пола,все население,,,,62.4,59.9,62.7,62.3,63.9,63.5,62.8,61.9,61.3,62.2,,,,,,,,,,,,,,,,,,,
Алтайский край,Оба пола,все население,68.9,68.4,67.7,65.0,64.0,64.3,65.3,66.0,67.8,66.8,66.6,66.3,65.6,65.7,65.5,64.6,66.5,67.1,67.4,68.4,68.4,68.97,69.11,69.77,70.01,70.44,70.74,71.1,71.11,71.61,70.19,68.6


In [5]:
def filter_regions(df_regions: 'dataframe', errors='raise'):
    """
    The function takes dataframe with regions, remove some rows and rename others.
    """
    
    ls_dropping = [
        'Центральный федеральный округ',
        'Северо-Западный федеральный округ',
        'Южный федеральный округ (по 2009 год)',
        'Южный федеральный округ (с 2010 года)',
        'Южный федеральный округ (с 29.07.2016)',
        'Северо-Кавказский федеральный округ',
        'Приволжский федеральный округ',
        'Уральский федеральный округ',
        'Сибирский федеральный округ',
        'Дальневосточный федеральный округ',
        'Крымский федеральный округ',
        'Коми-Пермяцкий округ, входящий в состав Пермского края',
        'Таймырский (Долгано-Ненецкий) автономный округ (Красноярский край)',
        'Эвенкийский автономный округ (Красноярский край)',
        'Усть-Ордынский Бурятский округ',
        'Агинский Бурятский округ (Забайкальский край)',
        'Корякский округ, входящий в состав Камчатского края',
        'Чеченская и Ингушская Республики' 
    ]

    dict_renaming = {
        'Российская Федерация': 'Россия в среднем',
        'Город Москва столица Российской Федерации город федерального значения': 'г. Москва',
        'Город Санкт-Петербург город федерального значения': 'г. Санкт-Петербург',
        'Республика Адыгея (Адыгея)': 'Республика Адыгея',
        'Город федерального значения Севастополь': 'г. Севастополь',
        'Республика Татарстан (Татарстан)': 'Республика Татарстан',
        'Чувашская Республика - Чувашия': 'Чувашская Республика (Чувашия)',
        'Ханты-Мансийский автономный округ - Югра (Тюменская область)': 'Ханты-Мансийский АО - Югра (Тюменская область)',
        'Ямало-Ненецкий автономный округ (Тюменская область)': 'Ямало-Ненецкий АО (Тюменская область)',
        'Ненецкий автономный округ (Архангельская область)': 'Ненецкий АО (Архангельская область)',
        'Тюменская область (кроме Ханты-Мансийского автономного округа-Югры и Ямало-Ненецкого автономного округа)': 'Тюменская область (кроме двух АО)',
        'Архангельская область (кроме Ненецкого автономного округа)': 'Архангельская область (кроме Ненецкого АО)',
        'Чукотский автономный округ': 'Чукотский АО'
    }
    
    return df_regions.drop(ls_dropping, errors=errors) \
                     .rename(index=dict_renaming, errors=errors) \
                     .sort_index()
    
df = filter_regions(df)
print(df.shape)
df.head(2)

(88, 34)


,Unnamed: 1,Unnamed: 2,1990,1991,1992,1993,1994,1995,1996,1997,1998,1999,2000,2001,2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Алтайский край,Оба пола,все население,68.9,68.4,67.7,65.0,64.0,64.3,65.3,66.0,67.8,66.8,66.6,66.3,65.6,65.7,65.5,64.6,66.5,67.1,67.4,68.4,68.4,68.97,69.11,69.77,70.01,70.44,70.74,71.10,71.11,71.61,70.19,68.6
Амурская область,Оба пола,все население,67.8,67.4,65.8,62.8,62.1,63.0,62.9,63.6,64.5,63.1,62.2,61.3,61.1,60.9,60.3,60.2,62.1,63.8,63.4,64.2,64.4,64.82,65.12,66.38,67.00,67.27,68.28,69.06,69.11,68.66,67.38,66.3


In [6]:
# select only required columns
df = df[['1991', '2000', '2014', '2019', '2020', '2021']]

print(df.shape)
df.head(2)

(88, 6)


,1991,2000,2014,2019,2020,2021
,,,,,,
Алтайский край,68.4,66.6,70.01,71.61,70.19,68.6
Амурская область,67.4,62.2,67.00,68.66,67.38,66.3


<br />
<br />
<br />

In [7]:
# load stats about longevity per gender ('genloc' is abbreviation for 'gender and location'
genloc = pd.read_excel('data/Rosstat_by_gender_and_location_2021.xls', skiprows=2)

print(genloc.shape)
genloc.head(10)

(864, 4)


,Unnamed: 0,Unnamed: 1,Unnamed: 2,2021
0,Российская Федерация,Женщины,все население,74.51
1,Российская Федерация,Женщины,городское население,74.69
2,Российская Федерация,Женщины,сельское население,73.88
3,Российская Федерация,Мужчины,все население,65.51
4,Российская Федерация,Мужчины,городское население,65.63
5,Российская Федерация,Мужчины,сельское население,65.12
6,Российская Федерация,Оба пола,все население,70.06
7,Российская Федерация,Оба пола,городское население,70.30
8,Российская Федерация,Оба пола,сельское население,69.31
9,Центральный федеральный округ,Женщины,все население,75.17


In [8]:
# rename columns
genloc.rename(columns={'Unnamed: 0': 'region', 'Unnamed: 1': 'gender', 'Unnamed: 2': 'location'}, inplace=True)

# remove surrounding spaces in the column with names of regions
genloc['region'] = genloc['region'].map(lambda st: st.strip())

genloc.head(10)

,region,gender,location,2021
0,Российская Федерация,Женщины,все население,74.51
1,Российская Федерация,Женщины,городское население,74.69
2,Российская Федерация,Женщины,сельское население,73.88
3,Российская Федерация,Мужчины,все население,65.51
4,Российская Федерация,Мужчины,городское население,65.63
5,Российская Федерация,Мужчины,сельское население,65.12
6,Российская Федерация,Оба пола,все население,70.06
7,Российская Федерация,Оба пола,городское население,70.30
8,Российская Федерация,Оба пола,сельское население,69.31
9,Центральный федеральный округ,Женщины,все население,75.17


In [9]:
# make dataframe multi-index
genloc = genloc.sort_values(by=['region', 'gender', 'location']) \
               .set_index(['region', 'gender', 'location'])

print(genloc.shape)
genloc.head(10)

(864, 1)


2021
region           gender   location                  
Алтайский край   Женщины  все население        73.10
                          городское население  73.71
                          сельское население   71.91
                 Мужчины  все население        64.08
                          городское население  64.43
                          сельское население   63.34
                 Оба пола все население        68.60
                          городское население  69.21
                          сельское население   67.49
Амурская область Женщины  все население        71.11

In [10]:
# filter records: remove some ot them and rename others
genloc = filter_regions(genloc, errors='ignore')

print(genloc.shape)
genloc.head(10)

(792, 1)


2021
region           gender   location                  
Алтайский край   Женщины  все население        73.10
                          городское население  73.71
                          сельское население   71.91
                 Мужчины  все население        64.08
                          городское население  64.43
                          сельское население   63.34
                 Оба пола все население        68.60
                          городское население  69.21
                          сельское население   67.49
Амурская область Женщины  все население        71.11

In [11]:
# reshape dataframe, transforming the last two levels of multi-index to columns
genloc = genloc.unstack().unstack()
genloc.head(2)

2021                                               \
location         все население                  городское население           
gender                 Женщины Мужчины Оба пола             Женщины Мужчины   
region                                                                        
Алтайский край           73.10   64.08     68.6               73.71   64.43   
Амурская область         71.11   61.75     66.3               71.78   62.18   

                                                               
location                  сельское население                   
gender           Оба пола            Женщины Мужчины Оба пола  
region                                                         
Алтайский край      69.21              71.91   63.34    67.49  
Амурская область    67.01              69.37   60.71    64.63

In [12]:
# simplify heading
heading = genloc.columns
genloc.columns = map(lambda tp: '-'.join(tp[1:]), heading)

genloc.head(2)

,все население-Женщины,все население-Мужчины,все население-Оба пола,городское население-Женщины,городское население-Мужчины,городское население-Оба пола,сельское население-Женщины,сельское население-Мужчины,сельское население-Оба пола
region,,,,,,,,,
Алтайский край,73.10,64.08,68.6,73.71,64.43,69.21,71.91,63.34,67.49
Амурская область,71.11,61.75,66.3,71.78,62.18,67.01,69.37,60.71,64.63


In [13]:
# choose only required to us columns and place them in required order
genloc = genloc[[
    'все население-Оба пола',
    'все население-Мужчины',
    'все население-Женщины',
    'городское население-Оба пола',
    'сельское население-Оба пола'    
]].rename(columns={
    'все население-Оба пола': 'overall',
    'все население-Мужчины': 'males',
    'все население-Женщины': 'females',
    'городское население-Оба пола': 'urban',
    'сельское население-Оба пола': 'rural'
})


# if in some column there's 0, than change it to NaN (it's actual for Saint Petersburg)
genloc.replace(0, np.nan, inplace=True)

genloc.head(2)

,overall,males,females,urban,rural
region,,,,,
Алтайский край,68.6,64.08,73.10,69.21,67.49
Амурская область,66.3,61.75,71.11,67.01,64.63


<br />

In [14]:
# before joining two dataframes make sure correctness of transformations
assert len(df) == len(genloc), "Problem wiht length of dataframes"
assert all(df.index == genloc.index), "Problem with indexes of dataframes"
assert all(df['2021'] == genloc['overall']), "content of the column '2021' is not identical to columns 'all_population'"

In [15]:
# concatinate info from two database
df = pd.concat([genloc, df], axis='columns')
df.head(2)

,overall,males,females,urban,rural,1991,2000,2014,2019,2020,2021
Алтайский край,68.6,64.08,73.10,69.21,67.49,68.4,66.6,70.01,71.61,70.19,68.6
Амурская область,66.3,61.75,71.11,67.01,64.63,67.4,62.2,67.00,68.66,67.38,66.3


In [16]:
# calculate difference between some columns and insert result as new columns in dataframe
df.insert(loc=3, column='females_vs_males', value=df['females']-df['males'])
df.insert(loc=6, column='urban_vs_rural', value=df['urban']-df['rural'])
df.insert(loc=8, column='1991→2000', value=df['2000']-df['1991'])
df.insert(loc=10, column='2000→2014', value=df['2014']-df['2000'])
df.insert(loc=12, column='2014→2019', value=df['2019']-df['2014'])
df.insert(loc=14, column='2019→2020', value=df['2020']-df['2019'])
df.insert(loc=16, column='2020→2021', value=df['2021']-df['2020'])
df.head(2)

,overall,males,females,females_vs_males,urban,rural,urban_vs_rural,1991,1991→2000,2000,2000→2014,2014,2014→2019,2019,2019→2020,2020,2020→2021,2021
Алтайский край,68.6,64.08,73.10,9.02,69.21,67.49,1.72,68.4,-1.8,66.6,3.41,70.01,1.60,71.61,-1.42,70.19,-1.59,68.6
Амурская область,66.3,61.75,71.11,9.36,67.01,64.63,2.38,67.4,-5.2,62.2,4.80,67.00,1.66,68.66,-1.28,67.38,-1.08,66.3


In [17]:
# sort table by 'all_population'
df.sort_values(by=['overall', '2020', '2019'], ascending=False, inplace=True)

In [18]:
# move some records to the top of dataframe (by default, column 'Россия в среднем')
# change order of some regions
indexes = df.index.to_list()
ls_change_order = ['Россия в среднем']    # 'г. Москва', 'г. Санкт-Петербург', 'г. Севастополь'
indexes = list(el for el in indexes if el not in ls_change_order)
indexes = ls_change_order + indexes
df = df.reindex(indexes)

<br />
<br />
<br />


In [19]:
popl = pd.read_excel('data/Rosstat_population_subjects_2021.xls', index_col=0, skiprows=7, usecols=[0, 7], nrows=96)  #, skiprows=2

# Clean names of regions from surrounding spaces
popl.index = popl.index.map(lambda st: st.strip())

# Chech that we downloaded all data, we can assume that the lowest LE in Chukotka
assert popl.iloc[-1].name == 'Чукотский автономный округ', 'Something wrong with reading of data'

popl.columns = ['pop']

print('Number of records:', len(popl))
popl.head(3)

Number of records: 96


,pop
Российская Федеpация,145864296
Центральный федеральный округ,39177680
Белгородская область,1536588


In [20]:
def filter_regions_popl(df_regions: 'dataframe', errors='raise'):
    """
    The function takes dataframe with regions, remove some rows and rename others.
    """
    
    ls_dropping = [
        'Центральный федеральный округ',
        'Северо-Западный федеральный округ',
        'Южный федеральный округ',
        'Северо-Кавказский федеральный округ',
        'Приволжский федеральный округ',
        'Уральский федеральный округ',
        'Сибирский федеральный округ',
        'Дальневосточный федеральный округ'
    ]

    dict_renaming = {
        'Российская Федеpация': 'Россия в среднем',
        'г.Санкт-Петербург': 'г. Санкт-Петербург',
        'Чувашская Республика': 'Чувашская Республика (Чувашия)',
        'Архангельская область без Ненецкого автономного округа': 'Архангельская область (кроме Ненецкого АО)',
        'Ненецкий автономный округ': 'Ненецкий АО (Архангельская область)',
        'Тюменская область без автономных округов': 'Тюменская область (кроме двух АО)',
        'Ханты-Мансийский автономный округ-Югра': 'Ханты-Мансийский АО - Югра (Тюменская область)',
        'Ямало-Ненецкий автономный округ': 'Ямало-Ненецкий АО (Тюменская область)',
        'Кемеровская область': 'Кемеровская область - Кузбасс',
        'Чукотский автономный округ': 'Чукотский АО'
    }
    
    return df_regions.drop(ls_dropping, errors=errors) \
                     .rename(index=dict_renaming, errors=errors) \
                     .sort_index()
    
popl = filter_regions_popl(popl)
print(popl.shape)
popl.head(3)

(88, 1)


,pop
Алтайский край,2282266
Амурская область,777185
Архангельская область,1120687


In [21]:
# round values and represent values in the form of thousands of people
popl['pop'] = popl['pop'].round(-3) // 1000
popl.head(3)

,pop
Алтайский край,2282
Амурская область,777
Архангельская область,1121


<br />
<br />
<br />


In [22]:
# before joining two dataframes make sure correctness of transformations
df_list = df.index.to_list()
popl_list = popl.index.to_list()

assert len(df_list) == len(popl_list), "Problem wiht length of dataframes"
assert [region for region in df_list if region not in popl_list] == [], "Problem with indexes of dataframes (1)"
assert [region for region in popl_list if region not in df_list] == [], "Problem with indexes of dataframes (2)"

In [23]:
# concatinate info from two database
df = pd.concat([df, popl], axis='columns')

In [24]:
ls_dropping = [
    'Архангельская область',
    'Тюменская область'
]

df_subjects = df.drop(index=ls_dropping)
print(df_subjects.shape)
df_subjects.fillna(' ')

(86, 19)


,overall,males,females,females_vs_males,urban,rural,urban_vs_rural,1991,1991→2000,2000,2000→2014,2014,2014→2019,2019,2019→2020,2020,2020→2021,2021,pop
Россия в среднем,70.06,65.51,74.51,9.00,70.30,69.31,0.99,68.9,-3.6,65.3,5.63,70.93,2.41,73.34,-1.80,71.54,-1.48,70.06,145864
Республика Ингушетия,80.52,77.26,83.29,6.03,81.67,79.24,2.43,,,72.0,7.42,79.42,3.98,83.40,-1.92,81.48,-0.96,80.52,520
Республика Дагестан,76.59,74.07,78.96,4.89,77.07,76.05,1.02,72.6,-1.6,71.0,4.83,75.83,3.27,79.10,-2.67,76.43,0.16,76.59,3144
г. Москва,74.55,71.00,77.94,6.94,74.65,68.55,6.1,69.8,0.0,69.8,6.9,76.70,1.66,78.36,-2.16,76.20,-1.65,74.55,12645
Кабардино-Балкарская Республика,73.77,69.91,77.32,7.41,74.10,73.36,0.74,70.5,-1.4,69.1,5.06,74.16,2.30,76.46,-2.09,74.37,-0.60,73.77,870
Карачаево-Черкесская Республика,73.47,69.28,77.35,8.07,73.42,73.48,-0.06,71.9,-3.5,68.4,5.51,73.91,2.30,76.21,-1.39,74.82,-1.35,73.47,465
Чеченская Республика,73.00,70.53,75.28,4.75,66.41,79.96,-13.55,69.8,,,,73.06,2.82,75.88,-3.77,72.11,0.89,73.00,1507
г. Санкт-Петербург,72.51,68.23,76.30,8.07,72.51,,,69.5,-2.8,66.7,7.87,74.57,1.74,76.31,-2.32,73.99,-1.48,72.51,5381
Республика Северная Осетия-Алания,72.47,68.08,76.46,8.38,72.19,73.02,-0.83,70.5,-2.1,68.4,5.42,73.82,1.93,75.75,-1.67,74.08,-1.61,72.47,691
г. Севастополь,72.25,67.87,76.43,8.56,72.13,73.9,-1.77,,,,,72.28,1.25,73.53,0.05,73.58,-1.33,72.25,516


In [25]:
df_subjects.head(1)

,overall,males,females,females_vs_males,urban,rural,urban_vs_rural,1991,1991→2000,2000,2000→2014,2014,2014→2019,2019,2019→2020,2020,2020→2021,2021,pop
Россия в среднем,70.06,65.51,74.51,9.0,70.3,69.31,0.99,68.9,-3.6,65.3,5.63,70.93,2.41,73.34,-1.8,71.54,-1.48,70.06,145864


<br />
<br />
<br />


In [26]:
dd_replacement_ru = {
    'Россия в среднем': 'Россия в среднем',
    'Республика Ингушетия': '[[Ингушетия]]',
    'Республика Дагестан': '[[Дагестан]]',
    'г. Москва': '[[Москва]]',
    'Кабардино-Балкарская Республика': '[[Кабардино-Балкария]]',
    'Карачаево-Черкесская Республика': '[[Карачаево-Черкесия]]',
    'Чеченская Республика': '[[Чечня]]',
    'г. Санкт-Петербург': '[[Санкт-Петербург]]',
    'Республика Северная Осетия-Алания': '[[Северная Осетия]]',
    'г. Севастополь': '[[Севастополь]]',
    'Ханты-Мансийский АО - Югра (Тюменская область)': '[[Ханты-Мансийский автономный округ — Югра|Ханты-Мансийский АО]]<br>(Тюменская область)',
    'Ямало-Ненецкий АО (Тюменская область)': '[[Ямало-Ненецкий автономный округ|Ямало-Ненецкий АО]]<br>(Тюменская область)',
    'Ставропольский край': '[[Ставропольский край]]',
    'Республика Калмыкия': '[[Калмыкия]]',
    'Республика Татарстан': '[[Татарстан]]',
    'Республика Адыгея': '[[Адыгея]]',
    'Калининградская область': '[[Калининградская область]]',
    'Белгородская область': '[[Белгородская область]]',
    'Краснодарский край': '[[Краснодарский край]]',
    'Московская область': '[[Московская область]]',
    'Республика Мордовия': '[[Мордовия]]',
    'Ленинградская область': '[[Ленинградская область]]',
    'Тюменская область (кроме двух АО)': '[[Тюменская область]]<br>(кроме двух АО)',
    'Чувашская Республика (Чувашия)': '[[Чувашия]]',
    'Удмуртская Республика': '[[Удмуртия]]',
    'Республика Саха (Якутия)': '[[Якутия|Саха (Якутия)]]',
    'Пензенская область': '[[Пензенская область]]',
    'Волгоградская область': '[[Волгоградская область]]',
    'Астраханская область': '[[Астраханская область]]',
    'Тамбовская область': '[[Тамбовская область]]',
    'Ростовская область': '[[Ростовская область]]',
    'Кировская область': '[[Кировская область]]',
    'Республика Крым': '[[Республика Крым]]',
    'Томская область': '[[Томская область]]',
    'Архангельская область (кроме Ненецкого АО)': '[[Архангельская область]]<br>(кроме АО)',
    'Воронежская область': '[[Воронежская область]]',
    'Республика Башкортостан': '[[Башкортостан]]',
    'Республика Марий Эл': '[[Марий Эл]]',
    'Ненецкий АО (Архангельская область)': '[[Ненецкий автономный округ|Ненецкий АО]]<br>(Архангельская область)',
    'Самарская область': '[[Самарская область]]',
    'Новосибирская область': '[[Новосибирская область]]',
    'Калужская область': '[[Калужская область]]',
    'Челябинская область': '[[Челябинская область]]',
    'Саратовская область': '[[Саратовская область]]',
    'Вологодская область': '[[Вологодская область]]',
    'Ярославская область': '[[Ярославская область]]',
    'Ульяновская область': '[[Ульяновская область]]',
    'Ивановская область': '[[Ивановская область]]',
    'Омская область': '[[Омская область]]',
    'Тульская область': '[[Тульская область]]',
    'Орловская область': '[[Орловская область]]',
    'Нижегородская область': '[[Нижегородская область]]',
    'Республика Бурятия': '[[Бурятия]]',
    'Свердловская область': '[[Свердловская область]]',
    'Костромская область': '[[Костромская область]]',
    'Брянская область': '[[Брянская область]]',
    'Рязанская область': '[[Рязанская область]]',
    'Приморский край': '[[Приморский край]]',
    'Алтайский край': '[[Алтайский край]]',
    'Липецкая область': '[[Липецкая область]]',
    'Курская область': '[[Курская область]]',
    'Пермский край': '[[Пермский край]]',
    'Республика Хакасия': '[[Хакасия]]',
    'Сахалинская область': '[[Сахалинская область]]',
    'Красноярский край': '[[Красноярский край]]',
    'Республика Коми': '[[Республика Коми|Коми]]',
    'Курганская область': '[[Курганская область]]',
    'Мурманская область': '[[Мурманская область]]',
    'Оренбургская область': '[[Оренбургская область]]',
    'Владимирская область': '[[Владимирская область]]',
    'Камчатский край': '[[Камчатский край]]',
    'Смоленская область': '[[Смоленская область]]',
    'Тверская область': '[[Тверская область]]',
    'Республика Алтай': '[[Республика Алтай|Алтай]]',
    'Хабаровский край': '[[Хабаровский край]]',
    'Псковская область': '[[Псковская область]]',
    'Новгородская область': '[[Новгородская область]]',
    'Кемеровская область - Кузбасс': '[[Кемеровская область|Кузбасс]]',
    'Магаданская область': '[[Магаданская область]]',
    'Республика Карелия': '[[Карелия]]',
    'Республика Тыва': '[[Тыва]]',
    'Забайкальский край': '[[Забайкальский край]]',
    'Иркутская область': '[[Иркутская область]]',
    'Амурская область': '[[Амурская область]]',
    'Еврейская автономная область': '[[Еврейская автономная область|Еврейская авт. область]]',
    'Чукотский АО': '[[Чукотский автономный округ|Чукотка]]',
    'Архангельская область': '[[Архангельская область]]',
    'Тюменская область': '[[Тюменская область]]'
}

### version with long names
# dd_replacement_ru = {
#     'Россия в среднем': 'Россия в среднем',
#     'Республика Ингушетия': '[[Ингушетия|Республика Ингушетия]]',
#     'Республика Дагестан': '[[Дагестан|Республика Дагестан]]',
#     'г. Москва': '[[Москва|г. Москва]]',
#     'Кабардино-Балкарская Республика': '[[Кабардино-Балкария|Кабардино-Балкарская Республика]]',
#     'Карачаево-Черкесская Республика': '[[Карачаево-Черкесия|Карачаево-Черкесская Республика]]',
#     'Чеченская Республика': '[[Чечня|Чеченская Республика]]',
#     'г. Санкт-Петербург': '[[Санкт-Петербург|г. Санкт-Петербург]]',
#     'Республика Северная Осетия-Алания': '[[Северная Осетия|Республика Северная Осетия — Алания]]',
#     'г. Севастополь': '[[Севастополь|г. Севастополь]]',
#     'Ханты-Мансийский АО - Югра (Тюменская область)': '[[Ханты-Мансийский автономный округ — Югра|Ханты-Мансийский АО — Югра]]<br>(Тюменская область)',
#     'Ямало-Ненецкий АО (Тюменская область)': '[[Ямало-Ненецкий автономный округ|Ямало-Ненецкий АО]]<br>(Тюменская область)',
#     'Ставропольский край': '[[Ставропольский край]]',
#     'Республика Калмыкия': '[[Калмыкия|Республика Калмыкия]]',
#     'Республика Татарстан': '[[Татарстан|Республика Татарстан]]',
#     'Республика Адыгея': '[[Адыгея|Республика Адыгея]]',
#     'Калининградская область': '[[Калининградская область]]',
#     'Белгородская область': '[[Белгородская область]]',
#     'Краснодарский край': '[[Краснодарский край]]',
#     'Московская область': '[[Московская область]]',
#     'Республика Мордовия': '[[Мордовия|Республика Мордовия]]',
#     'Ленинградская область': '[[Ленинградская область]]',
#     'Тюменская область (кроме двух АО)': '[[Тюменская область]]<br>(кроме двух АО)',
#     'Чувашская Республика (Чувашия)': '[[Чувашия|Чувашская Республика (Чувашия)]]',
#     'Удмуртская Республика': '[[Удмуртия|Удмуртская Республика]]',
#     'Республика Саха (Якутия)': '[[Якутия|Республика Саха (Якутия)]]',
#     'Пензенская область': '[[Пензенская область]]',
#     'Волгоградская область': '[[Волгоградская область]]',
#     'Астраханская область': '[[Астраханская область]]',
#     'Тамбовская область': '[[Тамбовская область]]',
#     'Ростовская область': '[[Ростовская область]]',
#     'Кировская область': '[[Кировская область]]',
#     'Республика Крым': '[[Республика Крым]]',
#     'Томская область': '[[Томская область]]',
#     'Архангельская область (кроме Ненецкого АО)': '[[Архангельская область]]<br>(кроме Ненецкого АО)',
#     'Воронежская область': '[[Воронежская область]]',
#     'Республика Башкортостан': '[[Башкортостан|Республика Башкортостан]]',
#     'Республика Марий Эл': '[[Марий Эл|Республика Марий Эл]]',
#     'Ненецкий АО (Архангельская область)': '[[Ненецкий автономный округ|Ненецкий АО]]<br>(Архангельская область)',
#     'Самарская область': '[[Самарская область]]',
#     'Новосибирская область': '[[Новосибирская область]]',
#     'Калужская область': '[[Калужская область]]',
#     'Челябинская область': '[[Челябинская область]]',
#     'Саратовская область': '[[Саратовская область]]',
#     'Вологодская область': '[[Вологодская область]]',
#     'Ярославская область': '[[Ярославская область]]',
#     'Ульяновская область': '[[Ульяновская область]]',
#     'Ивановская область': '[[Ивановская область]]',
#     'Омская область': '[[Омская область]]',
#     'Тульская область': '[[Тульская область]]',
#     'Орловская область': '[[Орловская область]]',
#     'Нижегородская область': '[[Нижегородская область]]',
#     'Республика Бурятия': '[[Бурятия|Республика Бурятия]]',
#     'Свердловская область': '[[Свердловская область]]',
#     'Костромская область': '[[Костромская область]]',
#     'Брянская область': '[[Брянская область]]',
#     'Рязанская область': '[[Рязанская область]]',
#     'Приморский край': '[[Приморский край]]',
#     'Алтайский край': '[[Алтайский край]]',
#     'Липецкая область': '[[Липецкая область]]',
#     'Курская область': '[[Курская область]]',
#     'Пермский край': '[[Пермский край]]',
#     'Республика Хакасия': '[[Хакасия|Республика Хакасия]]',
#     'Сахалинская область': '[[Сахалинская область]]',
#     'Красноярский край': '[[Красноярский край]]',
#     'Республика Коми': '[[Республика Коми]]',
#     'Курганская область': '[[Курганская область]]',
#     'Мурманская область': '[[Мурманская область]]',
#     'Оренбургская область': '[[Оренбургская область]]',
#     'Владимирская область': '[[Владимирская область]]',
#     'Камчатский край': '[[Камчатский край]]',
#     'Смоленская область': '[[Смоленская область]]',
#     'Тверская область': '[[Тверская область]]',
#     'Республика Алтай': '[[Республика Алтай]]',
#     'Хабаровский край': '[[Хабаровский край]]',
#     'Псковская область': '[[Псковская область]]',
#     'Новгородская область': '[[Новгородская область]]',
#     'Кемеровская область - Кузбасс': '[[Кемеровская область|Кемеровская область — Кузбасс]]',
#     'Магаданская область': '[[Магаданская область]]',
#     'Республика Карелия': '[[Карелия|Республика Карелия]]',
#     'Республика Тыва': '[[Тыва|Республика Тыва]]',
#     'Забайкальский край': '[[Забайкальский край]]',
#     'Иркутская область': '[[Иркутская область]]',
#     'Амурская область': '[[Амурская область]]',
#     'Еврейская автономная область': '[[Еврейская автономная область]]',
#     'Чукотский АО': '[[Чукотский автономный округ|Чукотский АО]]',
#     'Архангельская область': '[[Архангельская область]]',
#     'Тюменская область': '[[Тюменская область]]'
# }

In [27]:
# create code for placing info in Wikipedia
def create_table(df, file_header, file_footer, dd_replacement, lang='ru'):

    def if_value(x, prec=2):
        return '—' if math.isnan(x) else \
               f"{x:0.{prec}f}"  if x>=0 else \
               f"−{-x:0.{prec}f}"                #"{x:0.{prec}f}".format(x, prec)
    
    def chval(x, prec=2):  # change_value
        return 'style="background:#fffae0;"|—' if math.isnan(x) else \
               f'style="background:#fffae0;color:darkgreen;"|{x:0.{prec}f}' if x>0 else \
               f'style="background:#fffae0;color:crimson;"|−{-x:0.{prec}f}' if x<0 else \
               f'style="background:#fffae0;color:darkgray;"|−{x:0.{prec}f}'  # normal sorting is required '−' before 0.0 - bug in wiki
    
    def chval_bold(x, prec=2):  # change_value
        return '—' if math.isnan(x) else \
               f'style="background:#fffae0;color:darkgreen;"|\'\'\'{x:0.{prec}f}\'\'\'' if x>0 else \
               f'style="background:#fffae0;color:crimson;"|\'\'\'−{-x:0.{prec}f}\'\'\'' if x<0 else \
               f'style="background:#fffae0;color:darkgray;"|\'\'\'{x:0.{prec}f}\'\'\''
    
    with open('design/' + file_header, mode='r', encoding="utf-8") as fh:
        table_header = fh.read()
    
    with open('design/' + file_footer, mode='r', encoding="utf-8") as fh:
        table_footer = fh.read()

    st = ''
    for i in range(len(df)):
        ser = df.iloc[i]
        if ser.name == 'Россия в среднем':
            st += '\n' + '|-class=static-row-header\n' + \
                  f'|style="text-align:left;"|\'\'\'{dd_replacement[ser.name]}\'\'\' ' + \
                  f'|| style="text-align:center; background:#e0ffd8;"|\'\'\'{if_value(ser["overall"], 2)}\'\'\' ' + \
                  f'|| style="text-align:center;background:#eaf3ff;"|\'\'\'{if_value(ser["males"], 2)}\'\'\' ' + \
                  f'|| style="text-align:center;background:#fee7f6;"|\'\'\'{if_value(ser["females"], 2)}\'\'\' ' + \
                  f'|| style="background:#fff8dc;"|\'\'\'{if_value(ser["females_vs_males"], 2)}\'\'\' ' + \
                  f'|| style="text-align:center;border-left-width:2px;"|\'\'\'{if_value(ser["urban"], 2)}\'\'\' ' + \
                  f'|| style="text-align:center;"|\'\'\'{if_value(ser["rural"], 2)}\'\'\' ' + \
                  f'|| style="background:#fff8dc;"|\'\'\'{if_value(ser["urban_vs_rural"], 2)}\'\'\' ' + \
                  f'|| style="text-align:center;border-left-width:2px;"|\'\'\'{if_value(ser["1991"], 1)}\'\'\' ' + \
                  f'|| {chval_bold(ser["1991→2000"], 1)} ' + \
                  f'|| style="text-align:center;"|\'\'\'{if_value(ser["2000"], 1)}\'\'\' ' + \
                  f'|| {chval_bold(ser["2000→2014"], 2)} ' + \
                  f'|| style="text-align:center;"|\'\'\'{if_value(ser["2014"], 2)}\'\'\' ' + \
                  f'|| {chval_bold(ser["2014→2019"], 2)} ' + \
                  f'|| style="text-align:center;"|\'\'\'{if_value(ser["2019"], 2)}\'\'\' ' + \
                  f'|| {chval_bold(ser["2019→2020"], 2)} ' + \
                  f'|| style="text-align:center;"|\'\'\'{if_value(ser["2020"], 2)}\'\'\' ' + \
                  f'|| {chval_bold(ser["2020→2021"], 2)} ' + \
                  f'|| style="text-align:center; background:#e0ffd8;"|\'\'\'{if_value(ser["2021"], 2)}\'\'\' ' + \
                  f'|| style="color:dimgray;border-left-width:2px;"|\'\'\'{int(ser["pop"])}\'\'\''
        else:        
            st += '\n' + '|-\n' + \
                  f'|style="text-align:left;"|{dd_replacement[ser.name]} ' + \
                  f'|| style="text-align:center; background:#e0ffd8;"|\'\'\'{if_value(ser["overall"], 2)}\'\'\' ' + \
                  f'|| style="text-align:center;background:#eaf3ff;"|{if_value(ser["males"], 2)} ' + \
                  f'|| style="text-align:center;background:#fee7f6;"|{if_value(ser["females"], 2)} ' + \
                  f'|| style="background:#fff8dc;"|{if_value(ser["females_vs_males"], 2)} ' + \
                  f'|| style="text-align:center;border-left-width:2px;"|{if_value(ser["urban"], 2)} ' + \
                  f'|| style="text-align:center;"|{if_value(ser["rural"], 2)} ' + \
                  f'|| style="background:#fff8dc;"|{if_value(ser["urban_vs_rural"], 2)} ' + \
                  f'|| style="text-align:center;border-left-width:2px;"|{if_value(ser["1991"], 1)} ' + \
                  f'|| {chval(ser["1991→2000"], 1)} ' + \
                  f'|| style="text-align:center;"|{if_value(ser["2000"], 1)} ' + \
                  f'|| {chval(ser["2000→2014"], 2)} ' + \
                  f'|| style="text-align:center;"|{if_value(ser["2014"], 2)} ' + \
                  f'|| {chval(ser["2014→2019"], 2)} ' + \
                  f'|| style="text-align:center;"|{if_value(ser["2019"], 2)} ' + \
                  f'|| {chval(ser["2019→2020"], 2)} ' + \
                  f'|| style="text-align:center;"|{if_value(ser["2020"], 2)} ' + \
                  f'|| {chval(ser["2020→2021"], 2)} ' + \
                  f'|| style="text-align:center; background:#e0ffd8;"|\'\'\'{if_value(ser["2021"], 2)}\'\'\' ' + \
                  f'|| style="color:dimgray;border-left-width:2px;"|{int(ser["pop"])}'
            
    if lang == 'ru':
        st = re.sub('(?<=\d)\.(?=\d)', ',', st)  # replace . to comma, if this . is between two digits

    st = table_header + st + '\n' + table_footer
    
    # gray color for missing values
    st = st.replace(';"|—', ';color:silver;"|—')

    return st

table_code = create_table(df_subjects, file_header='rosstat_header_ru.txt', file_footer='rosstat_footer_ru.txt',
                          dd_replacement=dd_replacement_ru, lang='ru')

In [28]:
# write the code to file
with open('output/Table code for Russian regions -ru.txt', 'w', encoding="utf-8") as fh:
    fh.write(table_code)

<br />
<br />
<br />

In [29]:
dd_replacement_en = {
    'Россия в среднем': 'Russia on average',
    'Республика Ингушетия': '[[Ingushetia]]',
    'Республика Дагестан': '[[Dagestan]]',
    'г. Москва': '[[Moscow|Moscow]]',
    'Кабардино-Балкарская Республика': '[[Kabardino-Balkaria]]',
    'Карачаево-Черкесская Республика': '[[Karachay-Cherkessia]]',
    'Чеченская Республика': '[[Chechnya]]',
    'г. Санкт-Петербург': '[[Saint Petersburg]]',
    'Республика Северная Осетия-Алания': '[[North Ossetia]]',
    'г. Севастополь': '[[Sevastopol|Sevastopol]]',
    'Ханты-Мансийский АО - Югра (Тюменская область)': '[[Khanty-Mansi Autonomous Okrug|Khanty-Mansi AO]]<br>(Tyumen Oblast)',
    'Ямало-Ненецкий АО (Тюменская область)': '[[Yamalo-Nenets Autonomous Okrug|Yamalo-Nenets AO]]<br>(Tyumen Oblast)',
    'Ставропольский край': '[[Stavropol Krai]]',
    'Республика Калмыкия': '[[Kalmykia]]',
    'Республика Татарстан': '[[Tatarstan]]',
    'Республика Адыгея': '[[Adygea]]',
    'Калининградская область': '[[Kaliningrad Oblast]]',
    'Белгородская область': '[[Belgorod Oblast]]',
    'Краснодарский край': '[[Krasnodar Krai]]',
    'Московская область': '[[Moscow Oblast]]',
    'Республика Мордовия': '[[Mordovia]]',
    'Ленинградская область': '[[Leningrad Oblast]]',
    'Тюменская область (кроме двух АО)': '[[Tyumen Oblast]]<br>(except two AO)',
    'Чувашская Республика (Чувашия)': '[[Chuvashia]]',
    'Удмуртская Республика': '[[Udmurtia]]',
    'Республика Саха (Якутия)': '[[Sakha|Sakha (Yakutia)]]',
    'Пензенская область': '[[Penza Oblast]]',
    'Волгоградская область': '[[Volgograd Oblast]]',
    'Астраханская область': '[[Astrakhan Oblast]]',
    'Тамбовская область': '[[Tambov Oblast]]',
    'Ростовская область': '[[Rostov Oblast]]',
    'Кировская область': '[[Kirov Oblast]]',
    'Республика Крым': '[[Republic of Crimea]]',
    'Томская область': '[[Tomsk Oblast]]',
    'Архангельская область (кроме Ненецкого АО)': '[[Arkhangelsk Oblast]]<br>(except AO)',
    'Воронежская область': '[[Voronezh Oblast]]',
    'Республика Башкортостан': '[[Bashkortostan]]',
    'Республика Марий Эл': '[[Mari El]]',
    'Ненецкий АО (Архангельская область)': '[[Nenets Autonomous Okrug|Nenets AO]]<br>(Arkhangelsk Oblast)',
    'Самарская область': '[[Samara Oblast]]',
    'Новосибирская область': '[[Novosibirsk Oblast]]',
    'Калужская область': '[[Kaluga Oblast]]',
    'Челябинская область': '[[Chelyabinsk Oblast]]',
    'Саратовская область': '[[Saratov Oblast]]',
    'Вологодская область': '[[Vologda Oblast]]',
    'Ярославская область': '[[Yaroslavl Oblast]]',
    'Ульяновская область': '[[Ulyanovsk Oblast]]',
    'Ивановская область': '[[Ivanovo Oblast]]',
    'Омская область': '[[Omsk Oblast]]',
    'Тульская область': '[[Tula Oblast]]',
    'Орловская область': '[[Oryol Oblast]]',
    'Нижегородская область': '[[Nizhny Novgorod Oblast]]',
    'Республика Бурятия': '[[Buryatia]]',
    'Свердловская область': '[[Sverdlovsk Oblast]]',
    'Костромская область': '[[Kostroma Oblast]]',
    'Брянская область': '[[Bryansk Oblast]]',
    'Рязанская область': '[[Ryazan Oblast]]',
    'Приморский край': '[[Primorsky Krai]]',
    'Алтайский край': '[[Altai Krai]]',
    'Липецкая область': '[[Lipetsk Oblast]]',
    'Курская область': '[[Kursk Oblast]]',
    'Пермский край': '[[Perm Krai]]',
    'Республика Хакасия': '[[Khakassia]]',
    'Сахалинская область': '[[Sakhalin Oblast]]',
    'Красноярский край': '[[Krasnoyarsk Krai]]',
    'Республика Коми': '[[Komi Republic|Komi]]',
    'Курганская область': '[[Kurgan Oblast]]',
    'Мурманская область': '[[Murmansk Oblast]]',
    'Оренбургская область': '[[Orenburg Oblast]]',
    'Владимирская область': '[[Vladimir Oblast]]',
    'Камчатский край': '[[Kamchatka Krai]]',
    'Смоленская область': '[[Smolensk Oblast]]',
    'Тверская область': '[[Tver Oblast]]',
    'Республика Алтай': '[[Altai Republic|Altai]]',
    'Хабаровский край': '[[Khabarovsk Krai]]',
    'Псковская область': '[[Pskov Oblast]]',
    'Новгородская область': '[[Novgorod Oblast]]',
    'Кемеровская область - Кузбасс': '[[Kemerovo Oblast|Kuzbass]]',
    'Магаданская область': '[[Magadan Oblast]]',
    'Республика Карелия': '[[Republic of Karelia|Karelia]]',
    'Республика Тыва': '[[Tuva]]',
    'Забайкальский край': '[[Zabaykalsky Krai]]',
    'Иркутская область': '[[Irkutsk Oblast]]',
    'Амурская область': '[[Amur Oblast]]',
    'Еврейская автономная область': '[[Jewish Autonomous Oblast]]',
    'Чукотский АО': '[[Chukotka Autonomous Okrug|Chukotka]]'
}

### version with long names
# dd_replacement_en = {
#     'Россия в среднем': 'Russia on average',
#     'Республика Ингушетия': '[[Ingushetia|Republic of Ingushetia]]',
#     'Республика Дагестан': '[[Dagestan|Republic of Dagestan]]',
#     'г. Москва': '[[Moscow|federal city Moscow]]',
#     'Кабардино-Балкарская Республика': '[[Kabardino-Balkaria|Kabardino-Balkarian Republic]]',
#     'Карачаево-Черкесская Республика': '[[Karachay-Cherkessia|Karachay-Cherkess Republic]]',
#     'Чеченская Республика': '[[Chechnya|Chechen Republic]]',
#     'г. Санкт-Петербург': '[[Saint Petersburg|federal city Saint Petersburg]]',
#     'Республика Северная Осетия-Алания': '[[North Ossetia–Alania|Republic of North Ossetia–Alania]]',
#     'г. Севастополь': '[[Sevastopol|federal city Sevastopol]]',
#     'Ханты-Мансийский АО - Югра (Тюменская область)': '[[Khanty-Mansi Autonomous Okrug|Khanty-Mansi AO–Yugra]]<br>(Tyumen Oblast)',
#     'Ямало-Ненецкий АО (Тюменская область)': '[[Yamalo-Nenets Autonomous Okrug|Yamalo-Nenets AO]]<br>(Tyumen Oblast)',
#     'Ставропольский край': '[[Stavropol Krai]]',
#     'Республика Калмыкия': '[[Kalmykia|Republic of Kalmykia]]',
#     'Республика Татарстан': '[[Tatarstan|Republic of Tatarstan]]',
#     'Республика Адыгея': '[[Adygea|Republic of Adygea]]',
#     'Калининградская область': '[[Kaliningrad Oblast]]',
#     'Белгородская область': '[[Belgorod Oblast]]',
#     'Краснодарский край': '[[Krasnodar Krai]]',
#     'Московская область': '[[Moscow Oblast]]',
#     'Республика Мордовия': '[[Mordovia|Republic of Mordovia]]',
#     'Ленинградская область': '[[Leningrad Oblast]]',
#     'Тюменская область (кроме двух АО)': '[[Tyumen Oblast]]<br>(except two AO)',
#     'Чувашская Республика (Чувашия)': '[[Chuvashia|Chuvash Republic]]',
#     'Удмуртская Республика': '[[Udmurtia|Udmurt Republic]]',
#     'Республика Саха (Якутия)': '[[Sakha|Republic of Sakha (Yakutia)]]',
#     'Пензенская область': '[[Penza Oblast]]',
#     'Волгоградская область': '[[Volgograd Oblast]]',
#     'Астраханская область': '[[Astrakhan Oblast]]',
#     'Тамбовская область': '[[Tambov Oblast]]',
#     'Ростовская область': '[[Rostov Oblast]]',
#     'Кировская область': '[[Kirov Oblast]]',
#     'Республика Крым': '[[Republic of Crimea]]',
#     'Томская область': '[[Tomsk Oblast]]',
#     'Архангельская область (кроме Ненецкого АО)': '[[Arkhangelsk Oblast]]<br>(except Nenets AO])',
#     'Воронежская область': '[[Voronezh Oblast]]',
#     'Республика Башкортостан': '[[Bashkortostan|Republic of Bashkortostan]]',
#     'Республика Марий Эл': '[[Mari El|Mari El Republic]]',
#     'Ненецкий АО (Архангельская область)': '[[Nenets Autonomous Okrug|Nenets AO]]<br>(Arkhangelsk Oblast)',
#     'Самарская область': '[[Samara Oblast]]',
#     'Новосибирская область': '[[Novosibirsk Oblast]]',
#     'Калужская область': '[[Kaluga Oblast]]',
#     'Челябинская область': '[[Chelyabinsk Oblast]]',
#     'Саратовская область': '[[Saratov Oblast]]',
#     'Вологодская область': '[[Vologda Oblast]]',
#     'Ярославская область': '[[Yaroslavl Oblast]]',
#     'Ульяновская область': '[[Ulyanovsk Oblast]]',
#     'Ивановская область': '[[Ivanovo Oblast]]',
#     'Омская область': '[[Omsk Oblast]]',
#     'Тульская область': '[[Tula Oblast]]',
#     'Орловская область': '[[Oryol Oblast]]',
#     'Нижегородская область': '[[Nizhny Novgorod Oblast]]',
#     'Республика Бурятия': '[[Buryatia|Republic of Buryatia]]',
#     'Свердловская область': '[[Sverdlovsk Oblast]]',
#     'Костромская область': '[[Kostroma Oblast]]',
#     'Брянская область': '[[Bryansk Oblast]]',
#     'Рязанская область': '[[Ryazan Oblast]]',
#     'Приморский край': '[[Primorsky Krai]]',
#     'Алтайский край': '[[Altai Krai]]',
#     'Липецкая область': '[[Lipetsk Oblast]]',
#     'Курская область': '[[Kursk Oblast]]',
#     'Пермский край': '[[Perm Krai]]',
#     'Республика Хакасия': '[[Khakassia|Republic of Khakassia]]',
#     'Сахалинская область': '[[Sakhalin Oblast]]',
#     'Красноярский край': '[[Krasnoyarsk Krai]]',
#     'Республика Коми': '[[Komi Republic]]',
#     'Курганская область': '[[Kurgan Oblast]]',
#     'Мурманская область': '[[Murmansk Oblast]]',
#     'Оренбургская область': '[[Orenburg Oblast]]',
#     'Владимирская область': '[[Vladimir Oblast]]',
#     'Камчатский край': '[[Kamchatka Krai]]',
#     'Смоленская область': '[[Smolensk Oblast]]',
#     'Тверская область': '[[Tver Oblast]]',
#     'Республика Алтай': '[[Altai Republic]]',
#     'Хабаровский край': '[[Khabarovsk Krai]]',
#     'Псковская область': '[[Pskov Oblast]]',
#     'Новгородская область': '[[Novgorod Oblast]]',
#     'Кемеровская область - Кузбасс': '[[Kemerovo Oblast|Kemerovo Oblast–Kuzbass]]',
#     'Магаданская область': '[[Magadan Oblast]]',
#     'Республика Карелия': '[[Republic of Karelia]]',
#     'Республика Тыва': '[[Tuva|Tyva Republic]]',
#     'Забайкальский край': '[[Zabaykalsky Krai]]',
#     'Иркутская область': '[[Irkutsk Oblast]]',
#     'Амурская область': '[[Amur Oblast]]',
#     'Еврейская автономная область': '[[Jewish Autonomous Oblast]]',
#     'Чукотский АО': '[[Chukotka Autonomous Okrug|Chukotka AO]]'
# }

In [30]:
table_code = create_table(df_subjects, file_header='rosstat_header_en.txt', file_footer='rosstat_footer_en.txt',
                          dd_replacement=dd_replacement_en, lang='en')

In [31]:
# write the code to file
with open('output/Table code for Russian regions -en.txt', 'w', encoding="utf-8") as fh:
    fh.write(table_code)

<br />
<br />
<br />

#### Auxiliary table that I decided not to use

In [32]:
ls_attention = [
    'Архангельская область',
    'Архангельская область (кроме Ненецкого АО)',
    'Ненецкий АО (Архангельская область)',
    'Тюменская область',
    'Тюменская область (кроме двух АО)', 
    'Ханты-Мансийский АО - Югра (Тюменская область)',
    'Ямало-Ненецкий АО (Тюменская область)',
]

df_auxiliary = df.loc[ls_attention]
print(df_auxiliary.shape)
df_auxiliary.fillna(' ')

(7, 19)


,overall,males,females,females_vs_males,urban,rural,urban_vs_rural,1991,1991→2000,2000,2000→2014,2014,2014→2019,2019,2019→2020,2020,2020→2021,2021,pop
Архангельская область,69.60,64.32,75.09,10.77,70.49,65.51,4.98,69.1,-6.3,62.8,7.43,70.23,2.11,72.34,-0.99,71.35,-1.75,69.60,1121
Архангельская область (кроме Ненецкого АО),69.60,64.33,75.08,10.75,70.43,65.62,4.81,,,,,70.20,2.10,72.30,-0.91,71.39,-1.79,69.60,1076
Ненецкий АО (Архангельская область),69.39,64.18,75.01,10.83,71.84,62.49,9.35,,,60.6,10.05,70.65,2.54,73.19,-2.79,70.40,-1.01,69.39,44
Тюменская область,71.26,67.08,75.37,8.29,71.91,68.48,3.43,68.5,-2.6,65.9,5.6,71.50,2.54,74.04,-1.80,72.24,-0.98,71.26,3792
Тюменская область (кроме двух АО),70.14,65.43,74.88,9.45,70.98,68.36,2.62,,,,,70.32,2.43,72.75,-1.41,71.34,-1.20,70.14,1548
Ханты-Мансийский АО - Югра (Тюменская область),72.01,68.34,75.56,7.22,72.19,69.73,2.46,,,65.9,6.37,72.27,2.77,75.04,-2.17,72.87,-0.86,72.01,1695
Ямало-Ненецкий АО (Тюменская область),71.70,68.05,75.37,7.32,72.52,67.20,5.32,,,66.7,5.22,71.92,2.26,74.18,-2.27,71.91,-0.21,71.70,550


In [33]:
# create code for placing info in Wikipedia
def create_table_auxiliary(df, file_header):

    def if_value(x, prec=2):
        return '-' if math.isnan(x) else f"{x:0.{prec}f}" #"{x:0.{prec}f}".format(x, prec)
    
    with open('design/' + file_header, mode='r', encoding="utf-8") as fh:
        table_header = fh.read()

    # st = table_header
    st = table_header
    for i in range(len(df)):
        ser = df.iloc[i]       
        st += '\n' + '|-\n' + \
              f'|align=left|{dd_replacement_ru[ser.name]} ' + \
              f'|| style="text-align:center; background:#e0ffd8;"|\'\'\'{if_value(ser["overall"], 2)}\'\'\' ' + \
              f'|| style="background:#eaf3ff;"|{if_value(ser["males"], 2)} ' + \
              f'|| style="background:#fee7f6;"|{if_value(ser["females"], 2)} ' + \
              f'|| style="background:#fff8dc;"|{if_value(ser["females_vs_males"], 2)} ' + \
              f'|| style="border-left-width:2px;"|{if_value(ser["urban"], 2)} ' + \
              f'|| {if_value(ser["rural"], 2)} ' + \
              f'|| style="background:#fff8dc;"|{if_value(ser["urban_vs_rural"], 2)} ' + \
              f'|| style="border-left-width:2px;"|{if_value(ser["1991"], 1)} ' + \
              f'|| style="background:#fff8dc;"|{if_value(ser["1991→2000"], 1)} ' + \
              f'|| {if_value(ser["2000"], 1)} ' + \
              f'|| style="background:#fff8dc;"|{if_value(ser["2000→2014"], 2)} ' + \
              f'|| {if_value(ser["2014"], 2)} ' + \
              f'|| style="background:#fff8dc;"|{if_value(ser["2014→2019"], 2)} ' + \
              f'|| {if_value(ser["2019"], 2)} ' + \
              f'|| style="background:#fff8dc;"|{if_value(ser["2019→2020"], 2)} ' + \
              f'|| {if_value(ser["2020"], 2)} ' + \
              f'|| style="background:#fff8dc;"|{if_value(ser["2020→2021"], 2)} ' + \
              f'|| style="text-align:center; background:#e0ffd8;"|\'\'\'{if_value(ser["2021"], 2)}\'\'\' ' + \
              f'|| style="color:dimgray;border-left-width:2px;"|{int(ser["pop"])}'
    st += '\n' + '}}'
    return st

table_code = create_table_auxiliary(df_auxiliary, file_header='rosstat_header_ru.txt')
# print(table_code)

In [34]:
# write the code to file
# with open('output/Table code for Russian regions -attention.txt', 'w', encoding="utf-8") as fh:
#     fh.write(table_code)

# Naming of some regions are reqiered small manual asjustment for proper linking to Wikipedia pages of according redions.
# Ideally this should be make via dictionary - this code is not professional.

<br />
<br />
<br />

#### Additional task - forming data for [ZipMapMaker](https://navizv.github.io/zimm/)

In [35]:
# create data for server of colormapcreation ZiMapMaker (https://navizv.github.io/zimm)
df_colormap = df[['2019', '2020', '2021']] \
                    .drop(index='Россия в среднем')
print(df_colormap.shape)
df_colormap.head(3)

(87, 3)


,2019,2020,2021
Республика Ингушетия,83.40,81.48,80.52
Республика Дагестан,79.10,76.43,76.59
г. Москва,78.36,76.20,74.55


In [36]:
def filter_data_for_zipmapmaker_regions(df, errors='raise'):
    ls_dropping = [
        'Архангельская область',
        'Тюменская область'
    ]
    
    dict_renaming = {
        'Архангельская область (кроме Ненецкого АО)': 'Архангельская область (кроме АО)',
        'Ненецкий АО (Архангельская область)': 'Ненецкий автономный округ',        
        'Тюменская область (кроме двух АО)': 'Тюменская область (кроме двух АО)',
        'Ханты-Мансийский АО - Югра (Тюменская область)': 'Ханты-Мансийский автономный округ - Югра',
        'Ямало-Ненецкий АО (Тюменская область)': 'Ямало-Ненецкий автономный округ',
    }
    
    df_return = df.drop(ls_dropping, errors=errors) \
                  .rename(index=dict_renaming, errors=errors) \
                  [['2019', '2020', '2021']] \
                  .sort_values(by='2021', ascending=False)
    df_return.index.name = 'регион России'
    return df_return
    
data_for_zipmapmaker_regions = filter_data_for_zipmapmaker_regions(df_colormap)
data_for_zipmapmaker_regions

,2019,2020,2021
регион России,,,
Республика Ингушетия,83.40,81.48,80.52
Республика Дагестан,79.10,76.43,76.59
г. Москва,78.36,76.20,74.55
Кабардино-Балкарская Республика,76.46,74.37,73.77
Карачаево-Черкесская Республика,76.21,74.82,73.47
Чеченская Республика,75.88,72.11,73.00
г. Санкт-Петербург,76.31,73.99,72.51
Республика Северная Осетия-Алания,75.75,74.08,72.47
г. Севастополь,73.53,73.58,72.25


In [37]:
# data_for_zipmapmaker_regions.to_csv('output/data for ZipMapMaker colormap -regions.csv', sep=';', encoding='utf-8')
print('Done')

Done
